# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@arthur_juan***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [25]:
#    API KEY: ZpsQt4djTkQaRfHdIvwM4OVEQ
#    API Scret: qnAxY4HpajivQq76Gmz5uzks4Dp5AwMgZh5wopLI0vOYzEZs83
#
#    Access Token: 904543009957715968-zw2LsSLpUxRXol9z6FxatNjUAT01Wbh
#    Access Token Secret: dPULw1D3Q0fvF6CntlbTFvDdn3jO7MSK8e3ZiLfbNKdEJ

In [26]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @arthur_juan

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [27]:
#Produto escolhido:
produto = 'HTC Vive'
nome = "VIVE-EN"


#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [28]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [29]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(nome)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(nome))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [2]:
import pandas as pd
dados = pd.read_excel('VIVE-EN.xlsx')

#dados["Treinamento"]=

dados['Treinamento'] = dados['Treinamento'].map(lambda x: x.lstrip('rt'))


#dados['Treinamento'].str.replace('[":",";","[","]"]', '')



df = pd.DataFrame({"Treinamento":[]})
formatado=[]
formatado2=[]

for i in range(len(dados)):
    formatado.append(dados['Treinamento'][i].translate({ord(c): None for c in "@:;,[]“/{}()+=-_*."}))
    formatado2.append(dados['Relevância'][i])
df["Treinamento"]=formatado
df["Relevância"]=formatado2




#df['range'] = df['range'].str.replace(',','-')

df

,Treinamento,Relevância
0,"rickking16 watch ""rowdy joins the walking dea...",0
1,shahidkamal chimera zero for psvr oculus rift...,0
2,hese are the htc vive games launching in septe...,0
3,"""as someone who has been in the software indus...",1
4,hese are the htc vive games launching in septe...,0
5,the corridors of convergence” announced for oc...,0
6,i liked a youtube video httpstcoo7yr0rymnm sec...,0
7,cmoar vr un cine virtual para tu google cardbo...,0
8,26 vr startups debut at htc’s vive demo day ht...,0
9,dimension hunter vr gameplay htc vive oculus r...,0


In [3]:
pR = []
pIR = []


for i in range(len(dados)):
    if df["Relevância"][i] == 0:
        pIR.append(df['Treinamento'][i])
    else:
        pR.append(df['Treinamento'][i])


lpr = []
lpir = []


PR = (16/300)
PIR = (300 - 16)/300





for i in range(len(pR)):
    lpr += pR[i].split()

for i in range(len(pIR)):
    lpir += pIR[i].split()
    
#palaras_delete = ["a","i","o","on","who","has","in","it","now","as","me","same","day","too"]
#stopwords = ["htc",'vive','']




    
#print(lpr)
from collections import Counter

fr  = Counter(lpr)
fir = Counter(lpir)
print(fr)
#print(fir)

dicta = {}

dfqr = {}

for k,v in fr.items():
    dfqr[k] = v

    

dfqir = {}

for k,v in fir.items():
    dfqir[k] = v

    
#x = len(lpr)
#print(x)

#dfpr = pd.DataFrame()
#dfpi = pd.DataFrame()
#dfpr["R"]=lpr
#dfpi["IR"]=lpir

#dfpr.describe()



#for i in range(3):
#    print(fr[i]/250)
#print(len(lpir))
#print(fir) 
#print(dfqir)







Counter({'htc': 13, 'vive': 13, 'a': 7, 'i': 7, 'and': 6, 'the': 5, 'oculus': 4, 'vr': 4, 'has': 3, 'in': 3, 'amazing': 3, 'it': 3, 'on': 3, 'now': 3, '"as': 2, 'someone': 2, 'who': 2, 'been': 2, 'software': 2, 'industry': 2, 'for': 2, 'almost': 2, 'decade': 2, 'think': 2, 'it’s': 2, 'how': 2, 'good': 2, 'rift': 2, "it's": 2, 'to': 2, 'me': 2, 'that': 2, 'will': 2, 'passive': 2, '#vr': 2, '#virtualreality': 2, 'price': 2, 'htcvive': 2, 'are': 2, 'or': 2, 'crapping': 2, 'vive!': 2, 'my': 2, 'works"…': 1, 'httpstcot3om5itbmg': 1, 'bandainamcous': 1, 'playstationuk': 1, 'this': 1, 'shld': 1, 'be': 1, 'too': 1, 'clear': 1, 'shape': 1, 'shift': 1, 'experiences': 1, 'remarkable': 1, 'imagination': 1, 'he': 1, 'leaders': 1, 'headsets': 1, 'reduced': 1, 'by': 1, '$200': 1, 'award': 1, 'winning': 1, 'headset': 1, 'its': 1, '$75456…': 1, 'httpstcok2ac75aog8': 1, "don't": 1, 'miss': 1, 'gets': 1, 'major': 1, 'cut': 1, '$600': 1, 'httpstcou7vrrciekz': 1, 'agraylin': 1, 'obriend17': 1, 'httpstcomb1

In [4]:
#dfqr = {}

#for k,v in fr.items():
#    dfqr[k] = v

    

#dfqir = {}
#
#for k,v in fir.items():
#    dfqir[k] = v



print(fr)
print(fr[1])


Counter({'htc': 13, 'vive': 13, 'a': 7, 'i': 7, 'and': 6, 'the': 5, 'oculus': 4, 'vr': 4, 'has': 3, 'in': 3, 'amazing': 3, 'it': 3, 'on': 3, 'now': 3, '"as': 2, 'someone': 2, 'who': 2, 'been': 2, 'software': 2, 'industry': 2, 'for': 2, 'almost': 2, 'decade': 2, 'think': 2, 'it’s': 2, 'how': 2, 'good': 2, 'rift': 2, "it's": 2, 'to': 2, 'me': 2, 'that': 2, 'will': 2, 'passive': 2, '#vr': 2, '#virtualreality': 2, 'price': 2, 'htcvive': 2, 'are': 2, 'or': 2, 'crapping': 2, 'vive!': 2, 'my': 2, 'works"…': 1, 'httpstcot3om5itbmg': 1, 'bandainamcous': 1, 'playstationuk': 1, 'this': 1, 'shld': 1, 'be': 1, 'too': 1, 'clear': 1, 'shape': 1, 'shift': 1, 'experiences': 1, 'remarkable': 1, 'imagination': 1, 'he': 1, 'leaders': 1, 'headsets': 1, 'reduced': 1, 'by': 1, '$200': 1, 'award': 1, 'winning': 1, 'headset': 1, 'its': 1, '$75456…': 1, 'httpstcok2ac75aog8': 1, "don't": 1, 'miss': 1, 'gets': 1, 'major': 1, 'cut': 1, '$600': 1, 'httpstcou7vrrciekz': 1, 'agraylin': 1, 'obriend17': 1, 'httpstcomb1

In [15]:
palavrasR = len(lpr)
palavrasIR = len(lpir)
listu = Counter(lpr + lpir)

#print(listu)

print(fir)
print(fr)
print(1/(len(listu)+palavrasIR))

Counter({'htc': 253, 'vive': 246, 'vr': 128, 'the': 95, 'for': 78, 'on': 77, 'in': 71, 'and': 71, 'a': 69, 'rift': 67, 'oculus': 63, 'youtube': 59, '|': 50, 'of': 48, 'to': 47, 'virtual': 44, '#vr': 44, 'psvr': 42, 'i': 42, 'is': 40, 'video': 39, 'now': 33, 'this': 32, 'liked': 31, 'games': 29, 'your': 27, 'gameplay': 27, 'not': 23, 'demo': 22, 'september': 22, 'shahidkamal': 21, 'chimera': 21, 'zero': 21, 'tech': 21, 'currently': 21, 'runs': 21, 'daddy’s': 21, 'chim…': 21, 'available': 21, 'rickking16': 20, 'reality': 20, 'htcvive': 19, 'tpcast': 19, 'watch': 18, 'are': 18, 'preorders': 18, 'gameplay"': 17, 'via': 17, 'selling': 17, 'launching': 16, 'with': 16, 'new': 16, '&amp': 16, 'out': 15, 'cmoar': 15, 'un': 15, 'cine': 15, 'para': 15, 'tu': 15, 'google': 15, 'cardboard': 15, 'mindshow': 15, 'microsoft': 15, 'dead': 14, 'vr!': 14, 'headset': 14, 'raider': 14, 'na': 14, 'hese': 13, 'gets': 13, 'best': 13, 'an': 13, 'early': 13, 'walking': 12, 'ammo': 12, 'death': 12, 'drive': 12, 

In [8]:
listu = Counter(lpr + lpir)
total = len(listu)
palavrasR = len(lpr)
palavrasIR = len(lpir)

stringy = "htc htc htc htc htc htc htc htc htc htc htc htc htc htc htc"


listp = stringy.split()

x = PR
y = PIR


listR = [x]
listIR = [y]


for i in range(len(listp)):
    if listp[i] in lpr:
        listR.append((fr[listp[i]]+1)/(total+palavrasR))
        #print(listp[i] )
        #print(" RELEVANTE")
        #print(listR[i])
    else:
        listR.append(1/(total+palavrasR))
        #print("N TEM")
        #print(listR[i])
        
for i in range(len(listp)):
    if listp[i] in lpir:
        #print(dfqir[listp[i]]) 
        
        listIR.append((fir[listp[i]]+1)/(total+palavrasIR))
        #print(listp[i])
        #print(" IRRELEVANTE")
        #print(listIR[i])
    else:
        listIR.append(1/(total+palavrasIR))
        #print("N TEM")
        #print(listIR[i])
        
x = 1
y = 1

print(listR)
print(listIR)        
for i in range(len(listR)):
    x*=listR[i]
    y*=listIR[i]
    print(y)
    #print(x,"{}".format(i))
    #print(y)

    
#print(PRP)
#print(PIRP)

if x > y:
    print("Relevante")
else:
    print("Irrelevante")
print(x,y)

[0.05333333333333334, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772, 0.009824561403508772]
[0.9466666666666667, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994, 0.043853591160220994]
0.9466666666666667
0.04151473296500921
0.001820570126573263
7.983853800925567e-05
3.5012066046876623e-06
1.535404830094382e-07
6.733301568438762e-09
2.952794541407882e-10
1.294906445990335e-11
5.678629787319494e-13
2.4902830904336177e-14
1.0920785652108752e-15
4.789156691359846e-17
2.100217195451

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
